In [1]:
from binaryninja import *
import time
from Crypto.Cipher import AES
bv = BinaryViewType.get_view_of_file('usrnode.txt')
bv

<BinaryView: 'usrnode.txt', start 0x100000000, len 0x1c3e8>

In [2]:
ctx = PluginCommandContext(bv)
PluginCommand.get_valid_list(ctx)['Objc\\Run all'].execute(ctx)
while BackgroundTask.list:
    time.sleep(1)
print('Done!')

Done!


In [3]:
cfstring = bv.get_type_by_name('CFString')
info_offset = cfstring.structure['info'].offset
buffer_offset = cfstring.structure['buffer'].offset
br = BinaryReader(bv)
def get_buffer(cf):
    br.seek(cf+info_offset)
    info = br.read64()
    buffer = br.read64()
    length = br.read64() * (2 if info == 0x7d0 else 1)
    return bv.read(buffer, length)

In [5]:
def decrypt_AES_strings(caller, key):
    aes = AES.new(key, AES.MODE_ECB)
    for xref in bv.get_code_refs(caller.start):
        cf_ptr = xref.function.get_low_level_il_at(xref.address).mlil.params[2].constant
        br.seek(cf_ptr + buffer_offset)
        buffer = get_buffer(cf_ptr)
        
        decoded_buffer = Transform['Base64'].decode(buffer).encode('charmap')

        decrypted_buffer = aes.decrypt(decoded_buffer)
        print(f"Decrypted buffer: {decrypted_buffer}")

In [42]:
AESDecryptWithPassphrase = bv.get_symbols_by_name(
    '-[NSData AESDecryptWithPassphrase:]'
)[0]
for xref in bv.get_code_refs(AESDecryptWithPassphrase.address):
    xref_mlil = xref.function.get_low_level_il_at(xref.address).mlil
    caller = xref.function
    print(f"caller: {caller.name}")
    key_ptr = xref_mlil.params[2].constant
    print(f"key_ptr: {key_ptr:x}")
    
    key = get_buffer(key_ptr)
    key = key.decode('utf-16').encode('utf-8')
    key += b'\x00' * (32 - len(key))
    # key gets re-encoded as UTF-8, which is failing in python.
    # Damn demo gods!
    #key = b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92\x00\x00\x00\x00\x00\x00\x00\x00'
    print(f"Encryption key: {key}")
    decrypt_AES_strings(caller, key)

caller: -[AppDelegate yoop:]
key_ptr: 100013480
Encryption key: b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92\x00\x00\x00\x00\x00\x00\x00\x00'
Decrypted buffer: b'http://flux2key.com/liaROelcOeVvfjN/fsfSQNrIyxeRvXH.php?very=%@&xnvk=%@         '
Decrypted buffer: b'doc             '
Decrypted buffer: b'docx            '
Decrypted buffer: b'ppt             '
Decrypted buffer: b'pdf             '
Decrypted buffer: b'xls             '
Decrypted buffer: b'xlsx            '
Decrypted buffer: b'db              '
Decrypted buffer: b'txt             '
Decrypted buffer: b'rtf             '
Decrypted buffer: b'pptx            '
Decrypted buffer: b'string2me.com/qgHUDRZiYhOqQiN/kESklNvxsNZQcPl.php               '
Decrypted buffer: b'/Library        '
Decrypted buffer: b'tup             '
Decrypted buffer: b'http://flux2key.com/liaROelcOeVvfjN/fsfSQNrIyxeRvXH.php?very=%@&runs=%@&ertw=%@ '
Decrypted buffer: b'sow             '
Decrypted buffer: b'http://flux2key.com/liaR

In [30]:
b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92\x00\x00\x00\x00\x00\x00\x00\x00'.decode(
    'utf-8')

'æ$&łŁńŚŽ~Ę?|!~<Œ\x00\x00\x00\x00\x00\x00\x00\x00'

In [38]:
b'\xe6\x00$\x00&\x00B\x01A\x01D\x01Z\x01}\x01~\x00\x18\x01?\x00|\x00!\x00~\x00<\x00R\x01'.decode('utf-16').encode('utf-8')

b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92'

In [34]:
list(map(ord, _))

[230, 36, 38, 322, 321, 324, 346, 381, 126, 280, 63, 124, 33, 126, 60, 338]

In [35]:
list(map(ord, b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92\x00\x00\x00\x00\x00\x00\x00\x00'.decode(
    'utf-8')))

[230,
 36,
 38,
 322,
 321,
 324,
 346,
 381,
 126,
 280,
 63,
 124,
 33,
 126,
 60,
 338,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [36]:
'æ$&łŁńŚŽ~Ę?|!~<Œ'.encode('utf-8')

b'\xc3\xa6$&\xc5\x82\xc5\x81\xc5\x84\xc5\x9a\xc5\xbd~\xc4\x98?|!~<\xc5\x92'

In [37]:
len(_)

24